<a href="https://colab.research.google.com/github/Apoorva-Sharma-0012/Intelligent-video-Survellience-using-Deep-Learning/blob/main/violence_alert_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install telepot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.3 MB/s eta 0:00:00
  Created wheel for telepot: filename=telepot-12.7-py3-none-any.whl size=57944 sha256=a714fd898573e19eb789e1f0a7456900e5806e8d47abdf5debe0ff1b69d6684b
  Stored in directory: /root/.cache/pip/wheels/9f/9a/92/2e34a093e40a09338847e54dc9fcaab4ce01a59ba17c784c6c
Successfully built telepot


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.models import load_model
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import telepot
from datetime import datetime
import pytz

In [ ]:
def getTime():
  IST = pytz.timezone('Asia/Kolkata')
  timeNow = datetime.now(IST)
  return timeNow

In [ ]:
import numpy as np
import argparse
import pickle
import cv2
from google.colab.patches import cv2_imshow
import os
import time
from keras.models import load_model
from collections import deque

def print_results(video, limit=None):
        trueCount = 0
        imageSaved = 0
        filename = 'savedImage.jpg'
        finalImage = 'finaImage.jpg'
        sendAlert = 0
        location = "Delhi"

        print("Loading model ...")
        model = load_model('/content/drive/MyDrive/modelnew.h5')
        Q = deque(maxlen=128)
        vs = cv2.VideoCapture(video)
        writer = None
        (W, H) = (None, None)
        count = 0
        while True:

            (grabbed, frame) = vs.read()

            if not grabbed:
                break

            if W is None or H is None:
                (H, W) = frame.shape[:2]

            output = frame.copy()

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (128, 128)).astype("float32")
            frame = frame.reshape(128, 128, 3) / 255

            preds = model.predict(np.expand_dims(frame, axis=0))[0]
            Q.append(preds)

            results = np.array(Q).mean(axis=0)
            i = (preds > 0.50)[0]
            label = i

            text_color = (0, 255, 0)

            if label:
                text_color = (0, 0, 255)
                trueCount = trueCount + 1

            else:
                text_color = (0, 255, 0)

            text = "Violence: {}".format(label)
            FONT = cv2.FONT_HERSHEY_SIMPLEX

            cv2.putText(output, text, (35, 50), FONT,1.25, text_color, 3)

            if writer is None:
                fourcc = cv2.VideoWriter_fourcc(*"MJPG")
                writer = cv2.VideoWriter("recordedVideo.avi", fourcc, 30,(W, H), True)

            writer.write(output)

            cv2_imshow(output)

            if(trueCount == 50):
              if(imageSaved == 0):
                if(label):
                  cv2.imwrite(filename, output)
                  imageSaved = 1

              if(sendAlert == 0):
                timeMoment = getTime()
                bot = telepot.Bot('6141364366:AAEueP8EDriLyXQNVmroX0iwoiC_tPiVTC8')
                bot.sendMessage(-909837420, f"VIOLENCE ALERT!! \nLOCATION: {location} \nTIME: {timeMoment}")
                bot.sendPhoto(-909837420, photo=open(filename, 'rb'))
                sendAlert = 1

            key = cv2.waitKey(1) & 0xFF

            if key == ord("q"):
                break
        print("[INFO] cleaning up...")
        writer.release()
        vs.release()

In [ ]:
V_path = "/content/drive/MyDrive/Testing_videos/V_19.mp4"
NV_path = "/content/drive/MyDrive/Testing_videos/nonv.mp4"

In [ ]:
print_results(V_path)

In [ ]:
print_results(NV_path)